# Week 3

### Context
#### k-Fold Technic
- Label Postprocess only binary classification

#### Feature Selection
- Permutation Importance

#### AutoML
+ NNI

In [ ]:
import os
from os.path import join

import multiprocessing
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

n_cpus = multiprocessing.cpu_count()

In [ ]:
BASE_DIR = '../'

train_path = join(BASE_DIR, 'data', 'MDC14', 'train.csv')
test_path  = join(BASE_DIR, 'data', 'MDC14', 'test.csv')

data = pd.read_csv(train_path)
test = pd.read_csv(test_path)

label = data['credit']

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
test.head()

In [ ]:
test.describe()

In [ ]:
test.info()

In [ ]:
# 불필요한 컬럼 제거
data.drop(columns=['index', 'credit'], inplace=True)
test.drop(columns=['index'],           inplace=True)

In [ ]:
cat_columns = [c for c, t in zip(data.dtypes.index, data.dtypes) if t == 'O'] 
num_columns = [c for c    in data.columns if c not in cat_columns]

print('Categorical Columns: \n{}\n'.format(cat_columns))
print('Numeric Columns: \n{}'.format(num_columns))

#### 라벨 데이터 인코딩

In [ ]:
label = label.astype(int)

#### 전처리 프로세스 함수로 작성

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def preprocess(x_train, x_valid, x_test):
    tmp_x_train = x_train.copy()
    tmp_x_valid = x_valid.copy()
    tmp_x_test  = x_test.copy()
    
    tmp_x_train.reset_index(drop=True, inplace=True)
    tmp_x_valid.reset_index(drop=True, inplace=True)
    
    # 결측치 처리
    imputer = SimpleImputer(strategy='most_frequent')
    tmp_x_train[cat_columns] = imputer.fit_transform(tmp_x_train[cat_columns])
    tmp_x_valid[cat_columns] = imputer.transform(tmp_x_valid[cat_columns])
    tmp_x_test[cat_columns]  = imputer.transform(tmp_x_test[cat_columns])
    
    # 스케일링
    scaler = StandardScaler()
    tmp_x_train[num_columns] = scaler.fit_transform(tmp_x_train[num_columns])
    tmp_x_valid[num_columns] = scaler.transform(tmp_x_valid[num_columns])
    tmp_x_test[num_columns]  = scaler.transform(tmp_x_test[num_columns])

    # 인코딩
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(tmp_x_train[cat_columns])
    
    tmp_x_train_cat = pd.DataFrame(ohe.transform(tmp_x_train[cat_columns]))
    tmp_x_valid_cat = pd.DataFrame(ohe.transform(tmp_x_valid[cat_columns]))
    tmp_x_test_cat  = pd.DataFrame(ohe.transform(tmp_x_test[cat_columns]))
    
    tmp_x_train.drop(columns=cat_columns, inplace=True)
    tmp_x_valid.drop(columns=cat_columns, inplace=True)
    tmp_x_test.drop(columns=cat_columns, inplace=True)
    
    tmp_x_train = pd.concat([tmp_x_train, tmp_x_train_cat], axis=1)
    tmp_x_valid = pd.concat([tmp_x_valid, tmp_x_valid_cat], axis=1)
    tmp_x_test  = pd.concat([tmp_x_test, tmp_x_test_cat], axis=1)
    
    return tmp_x_train, tmp_x_valid, tmp_x_test

## k-Fold technic
### Label Postprocessing
- 이진 분류인 경우 Threshold 값을 최적화하여 조금 더 좋은 성능을 이끌어낼 수 있습니다. 
- k-Fold를 활용하여 train 라벨에서 가장 좋은 threshold 값을 찾아, 각 라벨 생성을 최적화 할 수 있습니다.

1. train 셋에 대해서도 라벨을 모읍니다.
2. y_train, y_train_pred 값으로 최적 threshold 값을 찾습니다.
3. threshold 를 0.01 단위로 0~1 사이값을 변경해가면서 평가 지표에 대해 평가합니다.
4. 그 중 가장 높은 평가 지표를 갖는 threshold를 선택합니다.

### 클래스가 0, 1인 데이터만 추출하겠습니다.

In [ ]:
binary_data = data.loc[label < 2,:]
binary_label = label.loc[label < 2]

In [ ]:
binary_data.reset_index(inplace=True, drop=True)
binary_label.reset_index(inplace=True, drop=True)

In [ ]:
binary_data.shape, binary_label.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics  import log_loss, accuracy_score

val_scores = list()
oof_train = np.zeros((binary_data.shape[0], 2))
oof_pred  = np.zeros((test.shape[0], 2))

for i, (trn_idx, val_idx) in enumerate(skf.split(binary_data, binary_label)):
    x_train, y_train = binary_data.iloc[trn_idx, :], binary_label.iloc[trn_idx,]
    x_valid, y_valid = binary_data.iloc[val_idx, :], binary_label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=n_cpus-1)
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 accuracy_score 확인
    trn_acc = accuracy_score(y_train, (model.predict_proba(x_train)[:, 1] > 0.5).astype(int))
    val_acc = accuracy_score(y_valid, (model.predict_proba(x_valid)[:, 1] > 0.5).astype(int))
    print('{} Fold, train accuracy_score : {:.4f}4, validation accuracy_score : {:.4f}'.format(i, trn_acc, val_acc))
    
    val_scores.append(val_acc)
    
    _, x_data, _ = preprocess(binary_data.iloc[trn_idx, :], binary_data, test)
    oof_train += model.predict_proba(x_data) / 5
    oof_pred  += model.predict_proba(x_test) / 5
    

# 교차 검증 accuracy_score 평균 계산하기
print('Cross Validation Score : {:.5f}'.format(np.mean(val_scores)))

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import f1_score

def threshold_search(y_true, y_proba):
    # 0 ~ 1 사이에서 0.01 단위로 threhold 생성
    thresholds = np.linspace(0, 1, 101) 

    # 각 threshold 마다 f1_score 측정합니다.
    f1_scores   = np.array([ f1_score(y_true, (y_proba > t).astype(np.int)) 
                             for t in thresholds ])
    best_score = np.max(f1_scores)

    # 가장 높은 f1_score를 갖는 threshold를 택합니다.
    best_th = thresholds[np.argmax(f1_scores)]
    return best_th, best_score

def scoring(y_true, y_proba, verbose=True):
    # 반복적으로 kFold를 진행할 수 있습니다.
    rkf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10)

    scores = []
    ths = []
    for train_index, test_index in rkf.split(y_true, y_true):
        y_prob_train, y_prob_test = y_proba[train_index], y_proba[test_index]
        y_true_train, y_true_test = y_true[train_index], y_true[test_index]

        best_threshold, sc = threshold_search(y_true_train, y_prob_train)
        
        ths.append(best_threshold)
        scores.append(sc)

    # 최적 threshold들의 평균 값으로 최종 threshold를 택합니다.
    best_th = np.mean(ths)
    score = np.mean(scores)

    if verbose: print(f'Best threshold: {np.round(best_th, 4)}, Score: {np.round(score, 5)}')

    return best_th, score

In [ ]:
best_th, _ = scoring(binary_label, oof_train[:, 1])

#### 찾은 threhold 값으로 Validation Score 재 측정

In [ ]:
val_scores = list()

for i, (trn_idx, val_idx) in enumerate(skf.split(binary_data, binary_label)):
    x_train, y_train = binary_data.iloc[trn_idx, :], binary_label.iloc[trn_idx,]
    x_valid, y_valid = binary_data.iloc[val_idx, :], binary_label.iloc[val_idx,]
    
    # 전처리
    x_train, x_valid, x_test = preprocess(x_train, x_valid, test)
    
    # 모델 정의
    model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, n_jobs=n_cpus-1)
    
    # 모델 학습
    model.fit(x_train, y_train)

    # 훈련, 검증 데이터 accuracy_score 확인
    trn_acc = accuracy_score(y_train, (model.predict_proba(x_train)[:, 1] > best_th).astype(int))
    val_acc = accuracy_score(y_valid, (model.predict_proba(x_valid)[:, 1] > best_th).astype(int))
    print('{} Fold, train accuracy_score : {:.4f}4, validation accuracy_score : {:.4f}'.format(i, trn_acc, val_acc))
    
    val_scores.append(val_acc)

# 교차 검증 accuracy_score 평균 계산하기
print('Cross Validation Score : {:.5f}'.format(np.mean(val_scores)))

## Feature Selection
### Permutation Importance
- 좋은 변수를 선택하는 방법 중 최근에 많이 사용되는 Permutation Importance에 대해 알아보겠습니다. 
- Permutation Importance의 기본 원리는 어떤 변수를 임의로 섞어 그 중 타겟 변수에 영향을 많이 주는 변수를 탐색합니다.
- 예를 들어 타겟 변수를 예측하기 좋은 변수는 임의로 섞었을 경우 모델의 성능이 많이 떨어지게되는데, 이러한 방식으로 좋은 변수를 선별해 냅니다.

$$ i_j=s-{{1}\over{K}}\sum^K_{k=1}s_{k,j}$$

$$ s=원본\ 데이터셋의\ 점수 [분류(Accuracy)| 회귀(R^2)] $$
$$ j=변수(feature)\ 인덱스$$
$$ k=반복\ 인덱스$$



#### ref
Permutation Importance: https://scikit-learn.org/stable/modules/permutation_importance.html

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(data, label, 
                                                      test_size=0.3,
                                                      random_state=42,
                                                      shuffle=True)

In [ ]:
x_train, x_valid, _ = preprocess(x_train, x_valid, test)

In [ ]:
from sklearn.inspection import permutation_importance

model = RandomForestClassifier(random_state=42, n_jobs=(n_cpus-1))
model.fit(x_train, y_train)

r = permutation_importance(model, x_valid, y_valid,
                           n_repeats=10,
                           random_state=42)

In [ ]:
for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{x_valid.columns[i]:<8}: "
               f"{r.importances_mean[i]:.3f}"
               f" +/- {r.importances_std[i]:.3f}")